In [1]:
%load_ext lineapy

In [2]:
import git
import pprint
import datetime
utc_timestamp = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
print('current UTC:', utc_timestamp)

current UTC: 20221013151359


# Sample of Validation Result

In this notebook, we demonstrate how to use `lineapy.get` to extract values and codes from other notebooks. 
Also, we demonstrate how to extract the metadata of you repo/branch and save it as an artifact.


## Extract repo information

In [3]:
repo = git.Repo()
repo_info = dict(
    repo_name = repo.working_tree_dir.split("/")[-1],
    repo_branch = repo.active_branch.name,
    repo_commit = str(repo.active_branch.commit),
    repo_diff = [{'a_path':x.a_path, 'b_path':x.b_path, 'change_type':x.change_type} for x in repo.index.diff(None)]
)
lineapy.save(repo_info,'repo_info')
pprint.pprint(repo_info)

{'repo_branch': 'validation_20221013142532',
 'repo_commit': '8cb4df2440950cc0d231f92b99f061678a42d518',
 'repo_diff': [{'a_path': 'cpp/cpp_workflow.ipynb',
                'b_path': 'cpp/cpp_workflow.ipynb',
                'change_type': 'M'},
               {'a_path': 'report_sample.ipynb',
                'b_path': 'report_sample.ipynb',
                'change_type': 'M'}],
 'repo_name': 'LineaPyModelValidation'}


## All Artifacts

In [4]:
lineapy.artifact_store()

excel_price_cell:0 created on 2022-10-13 02:39:47.190110
excel_price_cell:1 created on 2022-10-13 02:40:05.296699
excel_cell_price:0 created on 2022-10-13 03:02:36.321925
excel_cell_price:1 created on 2022-10-13 03:04:49.683406
excel_vba_price:0 created on 2022-10-13 03:05:36.452295
quantlib_config:0 created on 2022-10-13 03:39:27.644671
quantlib_version:0 created on 2022-10-13 03:39:28.830549
quantlib_price:0 created on 2022-10-13 03:39:31.865062
benchmark_quantlib_version:0 created on 2022-10-13 03:39:46.714295
benchmark_quantlib_price:0 created on 2022-10-13 03:39:47.218104
benchmark_quantlib_version:1 created on 2022-10-13 03:43:53.606886
benchmark_quantlib_price:1 created on 2022-10-13 03:43:54.131077
cpp_source_code:0 created on 2022-10-13 04:04:11.004027
cpp_config:0 created on 2022-10-13 04:04:12.436389
cpp_price:0 created on 2022-10-13 04:04:13.217499
R_source_code:0 created on 2022-10-13 04:10:09.173637
R_price:0 created on 2022-10-13 04:10:10.004397
excel_cell_price:2 create

## Model to Validate: `quantlib v1.25`

In [5]:
quantlib_version = lineapy.get('quantlib_version')
print(quantlib_version.get_value())

1.27


In [6]:
quantlib_price = lineapy.get('quantlib_price')
print(quantlib_price.get_value())

10.418271653959474


In [7]:
print(quantlib_price.get_code())

import datetime
import json
import sys

cpp_config_path = "../config/option_config.json"
params = json.load(open(cpp_config_path, "r"))


def load_quantlib(version="1.25"):
    sys.path.append(f"/home/mingjerli/quantlib/QuantLib-{version}")
    import QuantLib

    print(f"Loading QuantLib v{QuantLib.__version__}")
    return QuantLib


version = "1.27"
ql = load_quantlib(version)
today = datetime.datetime.today()
maturity_date = today + datetime.timedelta(365 * params["T"])
calculation_date = ql.Date(today.day, today.month, today.year + 1)
maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year + 1)


def quantlib_european_call(
    maturity_date=maturity_date,
    calculation_date=calculation_date,
    spot_price=params["S"],
    strike_price=params["K"],
    volatility=params["v"],
    risk_free_rate=params["r"],
):
    dividend_rate = 0
    option_type = ql.Option.Call

    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates()

    ql.Settings.

## Benchmark Model C++

In [8]:
cpp_price = lineapy.get('cpp_price')
print(cpp_price.get_value())

11.992892509814723


In [9]:
cpp_source_code = lineapy.get('cpp_config')
# print(cpp_source_code.get_value())
print(cpp_source_code.get_code())

import json

import pandas as pd
import sqlalchemy

calculation_date = pd.Timestamp.today().strftime("%Y-%m-%d")
eng = sqlalchemy.create_engine("sqlite:///../data/spot_price.db")
query = f"""SELECT spot FROM spotprice WHERE strftime('%Y-%m-%d',dt)='{calculation_date}' """
spot = pd.read_sql(query, con=eng).spot.iloc[0]
cpp_config_path = "../config/option_config.json"
params = json.load(open(cpp_config_path, "r"))
params["S"] = spot



In [10]:
cpp_source_code = lineapy.get('cpp_source_code')
# print(cpp_source_code.get_value())
print(cpp_source_code.get_code())

import pathlib
import subprocess

source_code_path = "lib.cpp"
shared_library_path = "./libTest.so"


def get_source_code(source_code_path, shared_library_path):
    p = subprocess.run(
        ["g++", "-fPIC", "-shared", "-o", shared_library_path, source_code_path]
    )
    source_code = pathlib.Path(source_code_path).read_text()
    return source_code


source_code = get_source_code(source_code_path, shared_library_path)



## Benchmark Model R `OptionPricing`

In [11]:
R_price = lineapy.get('R_price')
print(R_price.get_value())

10.4035391529966


In [12]:
R_source_code = lineapy.get('R_source_code')
print(R_source_code.get_value())
# print(R_source_code.get_code())

library(OptionPricing)
library(jsonlite)

config = fromJSON("../config/option_config.json")
european_call = BS_EC(K=config$K, r=config$r, sigma=config$v, T=config$T, S0=config$S)
print(european_call)
write.csv(european_call, file='R_call_price.csv')


## Benchmark Model quantlib v2

In [13]:
benchmark_quantlib_version = lineapy.get('benchmark_quantlib_version')
print(benchmark_quantlib_version.get_value())

1.25


In [14]:
benchmark_quantlib_price = lineapy.get('benchmark_quantlib_price')
print(benchmark_quantlib_price.get_value())

10.418271653959474


In [15]:
print(benchmark_quantlib_price.get_code())

quantlib_calculation = lineapy.get_function(
    ["quantlib_price", "quantlib_version"], input_parameters=["version"]
)
benchmark_quantlib_result = quantlib_calculation(version="1.25")
benchmark_quantlib_price = benchmark_quantlib_result["quantlib_price"]



## Benchmark Model Excel

In [16]:
excel_cell_price = lineapy.get('excel_cell_price')
print(excel_cell_price.get_value())

excel_vba_price = lineapy.get('excel_vba_price')
print(excel_vba_price.get_value())

10.403539152996622
10.403539152996622


# Combine all Results

Validators write their own comments

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame.from_records([
    {'name':'quantlib 1.27','modeltype':'target', 'value': quantlib_price.get_value()},
    {'name':'quantlib 1.25','modeltype':'bnenchmark', 'value': benchmark_quantlib_price.get_value()},
    {'name':'Monte Carlo C++','modeltype':'bnenchmark', 'value': cpp_price.get_value()},
    {'name':'R OptionPricing','modeltype':'bnenchmark', 'value': R_price.get_value()},
    {'name':'Excel Cell','modeltype':'bnenchmark', 'value': excel_cell_price.get_value()},
    {'name':'Excel VBA','modeltype':'bnenchmark', 'value': excel_vba_price.get_value()},
])

df

,name,modeltype,value
0,quantlib 1.27,target,10.418272
1,quantlib 1.25,bnenchmark,10.418272
2,Monte Carlo C++,bnenchmark,11.992893
3,R OptionPricing,bnenchmark,10.403539
4,Excel Cell,bnenchmark,10.403539
5,Excel VBA,bnenchmark,10.403539


## Save to a new branch and push

In [19]:
print(repo.git.status())

On branch validation_20221013142532
Your branch is up to date with 'origin/validation_20221013142532'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   cpp/cpp_workflow.ipynb
	modified:   report_sample.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	create_testing_data.ipynb
	data/

no changes added to commit (use "git add" and/or "git commit -a")


In [20]:
branch_name = f"validation_{utc_timestamp}"

In [21]:
print(repo.git.checkout("HEAD",b=branch_name))

M	cpp/cpp_workflow.ipynb
M	report_sample.ipynb


In [22]:
print(repo.git.status())

On branch validation_20221013151359
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   cpp/cpp_workflow.ipynb
	modified:   report_sample.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	create_testing_data.ipynb
	data/

no changes added to commit (use "git add" and/or "git commit -a")


In [23]:
print(repo.git.add("."))

In [24]:
print(repo.git.status())

On branch validation_20221013151359
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   cpp/cpp_workflow.ipynb
	new file:   create_testing_data.ipynb
	new file:   data/spot_price.db
	modified:   report_sample.ipynb



In [25]:
print(repo.git.commit(m=f"{utc_timestamp}_validation_report"))

[validation_20221013151359 10d9541] 20221013151359_validation_report
 4 files changed, 255 insertions(+), 43 deletions(-)
 create mode 100644 create_testing_data.ipynb
 create mode 100644 data/spot_price.db


In [26]:
print(repo.git.push("--set-upstream",repo.remote().name,branch_name))

Branch 'validation_20221013151359' set up to track remote branch 'validation_20221013151359' from 'origin'.
